<a href="https://colab.research.google.com/github/vsa1920/Football-Data/blob/main/Players.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
player_data = pd.read_csv("https://raw.githubusercontent.com/vsa1920/Football-Data/main/epl_players.csv")
match_data = pd.read_csv("https://raw.githubusercontent.com/vsa1920/Football-Data/main/epl_matches_train.csv")
potential_shots = pd.read_csv("https://raw.githubusercontent.com/vsa1920/Football-Data/main/epl_potential_shots.csv")
goals = pd.read_csv("https://raw.githubusercontent.com/vsa1920/Football-Data/main/epl_goals.csv")
player_data.drop(["birthday", "height", "weight", "preferred_foot"], axis=1, inplace=True)
player_data.dropna(inplace=True)

In [ ]:
match_data["date"] = pd.to_datetime(match_data["date"])
player_data["date"] = pd.to_datetime(player_data["date"])
player_temp = player_data.loc[pd.Timestamp("08/2014") < player_data["date"]]
player14_15 = player_temp[player_temp["date"] < pd.Timestamp("06/2015")]
player14_15.drop(["date"], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
player_set = pd.DataFrame(columns=player14_15.columns)

In [ ]:
unique_players = player14_15["player_id"].unique()
work_rate = {"high": 1, "medium": 0.5, "low": 0, "None": 0, "1": 0, "0": 0, "9": 0, "5": 0}

for p in range(len(unique_players)):
  pl = player14_15.loc[player14_15["player_id"] == unique_players[p]]
  for i in range(len(pl)):
    #print (pl['defensive_work_rate'].iloc[i], p, i)
    pl["attacking_work_rate"].iloc[i] = work_rate[pl['attacking_work_rate'].iloc[i]]
    pl["defensive_work_rate"].iloc[i] = work_rate[pl['defensive_work_rate'].iloc[i]]
  lst = [int(unique_players[p]),]
  for j in range(1, len(pl.columns)):
    lst.append(pl[pl.columns[j]].mean())
  player_set.loc[p] = lst



In [ ]:
player_set

In [ ]:
goals_temp = goals.loc[pd.Timestamp("08/2014") < goals["date"]]
goals14_15 = goals_temp.loc[pd.Timestamp("06/2015") > goals_temp["date"]]
goal_by_player = goals14_15["player1_id"].value_counts()
assist_by_player = goals14_15["player2_id"].value_counts()

In [ ]:
# Add number of goals score in season 14/15 to the player attributes
player_set["goals"] = 0
player_set["assists"] = 0
for p in range(len(goal_by_player)):
  player_set["goals"].loc[player_set["player_id"] == int(goal_by_player.keys()[p])] = int(goal_by_player.iloc[p])
for a in range(len(assist_by_player)):
  player_set["assists"].loc[player_set["player_id"] == int(assist_by_player.keys()[a])] = int(assist_by_player.iloc[a])

In [ ]:
player_set

,player_id,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,goals,assists
0,155782.0,0.5,0.5,77.750000,50.0,54.75,68.500000,28.0,66.000000,67.0,68.0,67.0,66.75,79.000000,82.000000,79.000000,70.50,90.0,70.0,84.00,79.0,50.0,56.0,66.0,64.25,57.0,64.000000,58.0,73.000000,72.000000,72.00,13.0,6.0,8.0,8.0,11.0,2,4
1,23780.0,0.5,0.5,45.000000,33.0,72.00,64.000000,32.0,46.000000,41.0,24.0,57.0,57.00,33.000000,31.000000,46.000000,67.75,59.0,47.0,73.75,60.5,74.0,25.0,65.0,71.25,28.0,40.000000,44.0,74.000000,70.750000,69.75,7.0,5.0,15.0,11.0,10.0,0,0
2,30895.0,1.0,0.5,78.333333,65.0,27.00,77.666667,67.0,83.000000,62.0,54.0,53.0,80.00,92.666667,89.666667,93.333333,77.00,92.0,61.0,71.00,75.0,52.0,56.0,56.0,36.00,74.0,75.666667,61.0,26.333333,26.333333,28.00,13.0,6.0,6.0,15.0,10.0,2,2
3,75489.0,1.0,1.0,74.000000,76.5,58.00,84.000000,79.5,81.000000,64.0,70.0,82.0,82.00,73.000000,69.000000,74.000000,81.00,75.0,81.0,63.00,93.0,69.5,78.0,75.0,72.00,84.0,81.000000,76.0,65.000000,75.500000,65.75,6.0,11.0,5.0,10.0,8.0,6,6
4,23499.0,0.5,0.5,48.000000,66.0,77.00,67.000000,55.0,48.000000,42.0,41.0,38.0,69.25,49.000000,56.000000,44.000000,66.00,52.0,68.0,67.00,68.0,81.0,46.0,74.0,23.00,74.0,67.000000,72.0,22.000000,25.000000,25.00,11.0,12.0,9.0,9.0,8.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,40825.0,0.5,0.5,82.000000,62.0,56.00,78.000000,66.0,77.000000,82.0,78.0,71.0,77.00,76.000000,78.000000,77.000000,70.00,71.0,72.0,57.00,81.0,67.0,75.0,83.0,73.00,76.0,80.000000,71.0,66.000000,72.000000,78.00,5.0,5.0,13.0,6.0,9.0,0,0
1137,33076.0,0.5,0.5,32.000000,26.0,74.00,54.000000,26.0,47.000000,34.0,41.0,53.0,55.00,44.000000,48.000000,31.000000,70.00,60.0,41.0,55.50,67.0,75.0,45.0,73.0,71.00,32.0,39.000000,36.0,66.000000,72.000000,65.00,7.0,13.0,11.0,15.0,7.0,0,0
1138,40128.0,0.0,0.0,25.000000,25.0,61.00,55.000000,30.0,40.000000,28.0,24.0,52.0,44.00,66.000000,68.000000,60.000000,52.00,60.0,40.0,70.00,70.0,74.0,25.0,40.0,56.00,31.0,31.000000,33.0,61.000000,70.000000,64.00,11.0,10.0,11.0,10.0,14.0,0,0
1139,119351.0,1.0,0.5,56.000000,57.0,40.00,58.000000,55.0,67.333333,49.0,50.0,46.0,62.00,90.000000,90.666667,90.000000,47.00,91.0,62.0,63.00,76.0,52.0,55.0,58.0,41.00,54.0,51.000000,50.0,25.000000,28.000000,25.00,8.0,9.0,15.0,5.0,7.0,0,0


Goalkeeper attributes
1) gk_diving
2) gk_handling
3) gk_kicking
4) gk_positioning
5) gk_reflexes

In [ ]:
# Best Goalkeepers
gk = pd.DataFrame(columns=["ID", "avg"])
gk["ID"] = player_set["player_id"]
gk["avg"] = (player_set['gk_reflexes'] + player_set['gk_positioning'] + player_set['gk_kicking'] + player_set['gk_handling'] + player_set['gk_diving']) / 5
gk_sorted = gk.sort_values(by=["avg"])
gk_sorted.iloc[-10:]

,ID,avg
782,37193.0,78.600000
1068,31465.0,79.500000
114,46518.0,80.266667
444,26295.0,81.266667
527,31432.0,81.300000
879,30859.0,81.400000
874,30660.0,81.500000
1098,32657.0,82.000000
1060,170323.0,82.800000
257,182917.0,84.160000


Attacking attributes


In [ ]:
# Best Attackers and Midfielders
# Max-goals: 27
# Max-assists: 18
ak1 = player_set.loc[player_set["goals"] + player_set["assists"] > 18]
ak1["goals+assists"] = ak1["goals"] + ak1["assists"]
ak1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,player_id,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,goals,assists,goals+assists
50,50047.0,1.000000,0.857143,75.571429,84.142857,60.000000,78.571429,77.857143,88.0,76.285714,74.428571,72.571429,86.142857,90.000000,85.142857,91.000000,82.714286,86.428571,80.285714,71.142857,83.142857,63.285714,80.000000,73.285714,42.0,82.142857,78.000000,73.000000,30.000000,39.000000,35.000000,10.0,10.0,15.0,12.0,13.0,17,8,25
174,30613.0,1.000000,0.500000,83.000000,78.000000,74.000000,91.500000,81.000000,81.0,77.000000,81.000000,91.000000,87.000000,67.000000,68.000000,65.000000,83.000000,77.000000,77.000000,68.000000,79.625000,64.000000,78.000000,45.000000,61.0,85.625000,92.500000,84.500000,62.000000,66.000000,62.000000,6.0,10.0,8.0,15.0,15.0,3,18,21
179,187266.0,1.000000,0.500000,44.444444,80.333333,78.888889,58.000000,67.111111,65.0,58.000000,42.000000,45.000000,72.000000,74.000000,74.888889,65.000000,77.444444,66.666667,74.888889,80.000000,76.222222,78.222222,68.222222,60.666667,46.0,79.444444,60.000000,74.888889,26.111111,27.444444,24.111111,8.0,10.0,6.0,8.0,12.0,21,6,27
272,37459.0,1.000000,0.000000,80.500000,74.500000,58.000000,88.500000,80.000000,87.0,83.000000,77.000000,85.000000,91.000000,82.750000,71.000000,93.000000,84.500000,88.000000,73.500000,66.000000,68.000000,53.000000,80.000000,51.000000,41.0,83.000000,90.000000,77.000000,23.000000,30.000000,29.000000,4.0,3.0,4.0,3.0,4.0,12,7,19
295,19243.0,1.000000,0.500000,63.000000,90.000000,82.400000,65.400000,81.000000,82.0,58.000000,59.000000,49.200000,82.000000,79.000000,85.000000,64.200000,85.000000,62.000000,82.400000,60.600000,85.000000,91.000000,69.000000,89.600000,40.0,88.600000,74.000000,76.000000,28.000000,39.000000,34.000000,11.0,13.0,12.0,8.0,11.0,20,3,23
312,107417.0,1.000000,0.500000,78.000000,83.000000,57.000000,85.600000,79.000000,92.8,82.000000,79.000000,82.000000,89.200000,93.000000,87.000000,92.000000,85.800000,90.000000,79.000000,59.000000,74.000000,63.000000,82.000000,54.000000,41.0,84.000000,86.000000,86.000000,25.000000,27.000000,22.000000,11.0,12.0,6.0,8.0,8.0,15,9,24
433,194165.0,0.933333,0.800000,65.800000,77.266667,75.200000,70.800000,65.933333,73.8,67.466667,69.133333,59.666667,75.333333,69.466667,75.200000,67.866667,72.466667,72.600000,75.200000,66.733333,78.133333,74.466667,70.666667,66.800000,31.0,80.000000,71.133333,71.600000,33.400000,30.400000,32.066667,7.0,9.0,10.0,13.0,10.0,24,4,28
987,37412.0,1.000000,0.166667,70.000000,89.777778,68.000000,84.000000,85.000000,89.0,82.000000,72.000000,63.000000,88.666667,92.000000,84.000000,86.000000,86.000000,90.000000,85.000000,76.000000,67.777778,68.888889,79.777778,57.000000,24.0,87.666667,83.000000,85.333333,25.000000,20.000000,25.000000,13.0,15.0,6.0,11.0,14.0,27,8,35


Defensive Attributes

defensive_work_rate \\
strength \\
interceptions \\
marking \\
standing_tackle \\
sliding_tackle 


In [ ]:
# Best Defenders
data = pd.DataFrame(columns=["ID","workrate", "avg"])
data["ID"] = player_set["player_id"]
data["workrate"] = player_set["defensive_work_rate"]
data["avg"] = (player_set["strength"] + player_set["interceptions"] + player_set["sliding_tackle"] + player_set["standing_tackle"] + player_set["marking"]) / 5
d_sorted = data.sort_values(by=["avg"])
d_sorted.iloc[-10:]
player_set.columns

Index(['player_id', 'attacking_work_rate', 'defensive_work_rate', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving',
       'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes', 'goals',
       'assists'],
      dtype='object')

0 goals and assists do not isolate the poor players
We also try and see players with all attributes less than very low thresholds


In [ ]:
# Worst Players
# Goalless/Assistless Players
# Contrary to good players poor players are bad at each part of the game
ng = player_set.loc[player_set["goals"] + player_set["assists"]==0]
weak_players = pd.DataFrame(columns=["ID",'workrate', "davg", "gavg", "rest"])
weak_players["ID"] = ng["player_id"]
weak_players["workrate"] = (ng['attacking_work_rate'] + ng['defensive_work_rate']) / 2 
weak_players["davg"] = (ng["strength"] + ng["interceptions"] + ng["sliding_tackle"] + ng["standing_tackle"] + ng["marking"]) / 5
weak_players["gavg"] = (ng['gk_reflexes'] + ng['gk_positioning'] + ng['gk_kicking'] + ng['gk_handling'] + ng['gk_diving']) / 5
attack = ng.drop(["strength", "interceptions", "sliding_tackle", "standing_tackle", "marking", 
                  "gk_reflexes", "gk_positioning", "gk_kicking", "gk_handling", "gk_diving", "goals", "assists", "attacking_work_rate", "defensive_work_rate", "player_id"], axis=1)
weak_players["rest"] = attack.mean(axis=1)

# Start eliminating players good at something
w = weak_players.loc[weak_players["gavg"]<40]
w1 = w.loc[w["davg"]<60]
w2 = w1.loc[w1["rest"]<60]
w2["tavg"] = (w2["davg"] + w2["rest"]) / 2
lowest = w2.sort_values(by=["tavg"])
lowest[:10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,workrate,davg,gavg,rest,tavg
901,750435.0,0.250000,23.400000,9.6,55.315217,39.357609
900,609734.0,0.750000,27.000000,11.2,56.478261,41.739130
287,492586.0,0.291667,25.533333,9.2,59.108696,42.321014
888,585352.0,0.500000,27.909091,6.6,58.308300,43.108696
307,520230.0,0.500000,30.450000,10.6,57.717391,44.083696
171,282276.0,0.750000,32.400000,12.8,57.130435,44.765217
999,608125.0,0.500000,35.000000,9.8,54.971014,44.985507
959,502421.0,0.500000,37.000000,11.0,54.521739,45.760870
800,509148.0,0.250000,34.400000,8.8,57.652174,46.026087
189,23431.0,0.250000,37.400000,8.6,55.086957,46.243478
